In [1]:
%%html
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

<html>
<body>


<dt>Thinkful Assignment</dt>
<dd>Unit 2 | Lesson 2 | Challenge 4</dd>

<ul>
    <li>Purpose: to show ability to draw logical inferences by visualizing data.</li>
    <li>Data: historical import and export data by country</li>
    <li>Source: the international monetary fund website</li>
    <li>By: AJ Igherighe | Aspiring Data Scientist</li>
    <li><b>Note:</b> This notebook focuses on cleaning up the data and saving the data for visualization</li>
</ul>
<br>
<b>Assignment Milestones</b>
<ul>
    <li> Select a dataset</li>
    <li> Clean up data</li>
    <li> Create a series of visualizations (minimum of 4)</li>
    <li> Infer what each visualization says about the dataset</li>
</ul>

</body>
</html>

<html>
    <body>
        <dt>Import modules used in analysis</dt>
        <ul>
            <li>Pandas and numpy for data analysis</li>
            <li>Matplotlib and seaborn for visualization</li>
            <li>Other items to improve functionality of notebook</li>
        </ul>
    </body>
</html>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn
import copy

# display dataframe head and tail views as formatted tables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

<html>
    <body>
        <dt>Create dataframe with original data</dt>
        <ul>
            <li>Create a function that returns a copy of the in-function dataframe</li>
            <li>Use .head() to get a quick view of the data to determine what to clean</li>
        </ul>
    </body>
</html>

In [3]:
def create_dataframe(csv_file):
    """read a supplied csv file and returns a pandas dataframe"""
    try:
        data = pd.read_csv(csv_file)
        return data.copy()
    except:
        print('Error reading csv file into dataframe.')

file_path = './Data/ImportExport.csv'
data_original = create_dataframe(file_path)
data_original.head()

,Country or Area,Year,Comm. Code,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity
0,Afghanistan,2016,TOTAL,ALL COMMODITIES,Import,6534140413,NaN,No Quantity,NaN
1,Afghanistan,2016,TOTAL,ALL COMMODITIES,Export,596455337,NaN,No Quantity,NaN
2,Afghanistan,2015,TOTAL,ALL COMMODITIES,Import,7722865049,NaN,No Quantity,NaN
3,Afghanistan,2015,TOTAL,ALL COMMODITIES,Export,571404967,NaN,No Quantity,NaN
4,Afghanistan,2014,TOTAL,ALL COMMODITIES,Import,7697178170,NaN,No Quantity,NaN


<html>
    <body>
        <dt>Prepare data for visualization</dt>
        <ul>
            <li>Remove unwanted columns</li>
            <li>Rename columns to make analysis easier</li>
            <li>Change Re-Import and Re-Export to Import and Export</li>
            <li>Make all Import flows have negative values in "Trade_ADJ" column</li>
            <li>Set 'Year' column as the dataframe index and delete it</li>
            <li>Save cleaned dataframe to make it easier to access going forward</li>
        </ul>
    </body>
</html>

In [4]:
data_original.drop(data_original.columns[[2,3,6,7,8]], axis=1, inplace=True)
data_original.head()

data_original.columns = ['COUNTRY', 'YEAR', 'FLOW', 'TRADE']
data_original.head()

,Country or Area,Year,Flow,Trade (USD)
0,Afghanistan,2016,Import,6534140413
1,Afghanistan,2016,Export,596455337
2,Afghanistan,2015,Import,7722865049
3,Afghanistan,2015,Export,571404967
4,Afghanistan,2014,Import,7697178170


,COUNTRY,YEAR,FLOW,TRADE
0,Afghanistan,2016,Import,6534140413
1,Afghanistan,2016,Export,596455337
2,Afghanistan,2015,Import,7722865049
3,Afghanistan,2015,Export,571404967
4,Afghanistan,2014,Import,7697178170


In [5]:
flows = copy.deepcopy(pd.Series(data_original['FLOW'], index=data_original.index))

for index, value in flows.iteritems():
    if value == 'Re-Import':
        flows[index] = 'Import'
    elif value == 'Re-Export':
        flows[index] = 'Export'

data_original['FLOW_ADJ'] = flows
data_original.head()
data_original.tail()

,COUNTRY,YEAR,FLOW,TRADE,FLOW_ADJ
0,Afghanistan,2016,Import,6534140413,Import
1,Afghanistan,2016,Export,596455337,Export
2,Afghanistan,2015,Import,7722865049,Import
3,Afghanistan,2015,Export,571404967,Export
4,Afghanistan,2014,Import,7697178170,Import


,COUNTRY,YEAR,FLOW,TRADE,FLOW_ADJ
9180,Zimbabwe,2001,Export,1206807424,Export
9181,Zimbabwe,2001,Re-Export,15943977,Export
9182,Zimbabwe,2000,Export,1924962432,Export
9183,Zimbabwe,1995,Import,2658853376,Import
9184,Zimbabwe,1995,Export,1845905792,Export


In [6]:
# confirm we only have Export and Import in the new column
print(data_original.groupby('FLOW_ADJ').describe())

           TRADE                                                  \
           count          mean           std   min           25%   
FLOW_ADJ                                                           
Export    4839.0  6.085637e+10  1.893284e+11  48.0  2.347007e+08   
Import    4346.0  6.673944e+10  2.126641e+11   2.0  8.416611e+08   

                                                      YEAR               \
                   50%           75%           max   count         mean   
FLOW_ADJ                                                                  
Export    2.381132e+09  2.951183e+10  2.342293e+12  4839.0  2004.753461   
Import    5.228345e+09  3.434607e+10  2.443311e+12  4346.0  2005.181316   

                                                            
               std     min     25%     50%     75%     max  
FLOW_ADJ                                                    
Export    6.942224  1988.0  1999.0  2005.0  2011.0  2016.0  
Import    6.830196  1988.0  2000.0  200

In [7]:
# convert all imports to negative numbers
flows = copy.deepcopy(pd.Series(data_original['FLOW_ADJ'], index=data_original.index))
trade = copy.deepcopy(pd.Series(data_original['TRADE'], index=data_original.index))

for index, value in flows.iteritems():
    if value == 'Import':
        trade[index] *= -1

data_original['TRADE_ADJ'] = trade
data_original.head()
data_original.tail()

,COUNTRY,YEAR,FLOW,TRADE,FLOW_ADJ,TRADE_ADJ
0,Afghanistan,2016,Import,6534140413,Import,-6534140413
1,Afghanistan,2016,Export,596455337,Export,596455337
2,Afghanistan,2015,Import,7722865049,Import,-7722865049
3,Afghanistan,2015,Export,571404967,Export,571404967
4,Afghanistan,2014,Import,7697178170,Import,-7697178170


,COUNTRY,YEAR,FLOW,TRADE,FLOW_ADJ,TRADE_ADJ
9180,Zimbabwe,2001,Export,1206807424,Export,1206807424
9181,Zimbabwe,2001,Re-Export,15943977,Export,15943977
9182,Zimbabwe,2000,Export,1924962432,Export,1924962432
9183,Zimbabwe,1995,Import,2658853376,Import,-2658853376
9184,Zimbabwe,1995,Export,1845905792,Export,1845905792


In [8]:
data_original.index = data_original['YEAR']
del data_original['YEAR']
data_original.head()

,COUNTRY,FLOW,TRADE,FLOW_ADJ,TRADE_ADJ
YEAR,,,,,
2016,Afghanistan,Import,6534140413,Import,-6534140413
2016,Afghanistan,Export,596455337,Export,596455337
2015,Afghanistan,Import,7722865049,Import,-7722865049
2015,Afghanistan,Export,571404967,Export,571404967
2014,Afghanistan,Import,7697178170,Import,-7697178170


###### Saving file in .pickle format
Will retrieve in visualization notebook

In [9]:
dataframe_filename = 'originaldataframe.pickle'
data_original.to_pickle(dataframe_filename)